<a href="https://colab.research.google.com/github/lcqsigi/big-data2/blob/main/063_Zillow_For_Sale_Properties_Python2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Steps
### Get Keys
1) Get Free [Rapid API Key by creating an account](https://rapidapi.com/marketplace)

Store each key in a central and private location.

### Subscribe to APIs

1) Subscribe to [Zillow on Rapid API](https://rapidapi.com/apimaker/api/zillow-com1/)

## <font color="blue">Install Packages</font>

## <font color="blue">Imports</font>

In [1]:
from google.colab import drive, files # specific to Google Colab
import pandas as pd
import requests
import json
import time

# show all columns
pd.set_option('display.max_columns', None)

## <font color="blue">Functions</font>

## <font color="blue">Local & Constants</font>

In [2]:
############
# OPTIONAL #
############

# mount drive
drive.mount('/content/drive', force_remount=False)

# data location
file_dir = '/content/drive/My Drive/' # optional

############
# REQURIED #
############
city = 'moraga'
state = 'ca'
search_str = city + ', ' + state
print('Search string:', search_str)

Mounted at /content/drive
Search string: moraga, ca


## <font color="blue">Data</font>

### Get API Keys

In [3]:
# read in api key file
df_api_keys = pd.read_csv(file_dir + 'api_keys.csv')

# get keys
rapid_api_key = df_api_keys.loc[df_api_keys['API'] =='rapid']['KEY'].iloc[0] # replace this with your own key

In [4]:
rapid_api_key

'8b63d59cecmsha5a796447533ec2p11f828jsnf7fd2c8395a5'

In [5]:
url = "https://zillow-com1.p.rapidapi.com/propertyExtendedSearch"

In [6]:
headers = {
    'x-rapidapi-host': "zillow-com1.p.rapidapi.com",
    'x-rapidapi-key': rapid_api_key
    }
import requests
req = requests.request('GET', url, headers=headers)

## <font color="green">Zillow</font>

### <font color="purple">Endpoint: Property</font>

In [7]:
# get data
url = "https://zillow-com1.p.rapidapi.com/propertyExtendedSearch"

querystring = {"location":search_str,
               "home_type":"Condos",
               "minPrice": "700000",
               "maxPrice": "9000000",
               "sqftMin": "1000",
               "bedsMin": "2"
               }

headers = {
    'x-rapidapi-host': "zillow-com1.p.rapidapi.com",
    'x-rapidapi-key': rapid_api_key
    }

z_for_sale_resp = requests.request("GET", url, headers=headers, params=querystring)

# transform to json
z_for_sale_resp_json = z_for_sale_resp.json()
z_for_sale_resp_json

{'props': [{'dateSold': None,
   'propertyType': 'CONDO',
   'lotAreaValue': 1306.8,
   'address': '54 Miramonte Dr, Moraga, CA 94556',
   'variableData': {'text': 'Open: Sun. 2-4pm', 'type': 'OPEN_HOUSE'},
   'priceChange': -20000,
   'zestimate': None,
   'imgSrc': 'https://photos.zillowstatic.com/fp/9b9f3841d236fe7c489394bb725e7859-p_e.jpg',
   'price': 859000,
   'detailUrl': '/homedetails/54-Miramonte-Dr-Moraga-CA-94556/18471532_zpid/',
   'bedrooms': 2,
   'contingentListingType': None,
   'longitude': -122.144135,
   'latitude': 37.839676,
   'listingStatus': 'FOR_SALE',
   'zpid': '18471532',
   'listingSubType': {'is_FSBA': True, 'is_openHouse': True},
   'rentZestimate': 3951,
   'daysOnZillow': 95,
   'bathrooms': 2,
   'livingArea': 1248,
   'country': 'USA',
   'currency': 'USD',
   'lotAreaUnit': 'sqft',
   'hasImage': True}],
 'resultsPerPage': 41,
 'totalPages': 1,
 'totalResultCount': 1,
 'schools': {},
 'currentPage': 1}

In [8]:
# view data
df_z_for_sale = pd.json_normalize(data=z_for_sale_resp_json['props'])
print('Num of rows:', len(df_z_for_sale))
print('Num of cols:', len(df_z_for_sale.columns))
df_z_for_sale.head()

Num of rows: 1
Num of cols: 27


,dateSold,propertyType,lotAreaValue,address,priceChange,zestimate,imgSrc,price,detailUrl,bedrooms,contingentListingType,longitude,latitude,listingStatus,zpid,rentZestimate,daysOnZillow,bathrooms,livingArea,country,currency,lotAreaUnit,hasImage,variableData.text,variableData.type,listingSubType.is_FSBA,listingSubType.is_openHouse
0,None,CONDO,1306.8,"54 Miramonte Dr, Moraga, CA 94556",-20000,None,https://photos.zillowstatic.com/fp/9b9f3841d23...,859000,/homedetails/54-Miramonte-Dr-Moraga-CA-94556/1...,2,None,-122.144135,37.839676,FOR_SALE,18471532,3951,95,2,1248,USA,USD,sqft,True,Open: Sun. 2-4pm,OPEN_HOUSE,True,True


In [9]:
# download file


In [10]:
# get zpids to a list
zpid_list = df_z_for_sale['zpid'].tolist()
zpid_list

['18471532']

In [11]:
# get property detail

# create empty list
prop_detail_list = []

# iterate through list of properties
for zpid in zpid_list:

  # end point
  url = "https://zillow-com1.p.rapidapi.com/property"

  querystring = {"zpid":zpid}

  # header
  headers = {
      'x-rapidapi-host': "zillow-com1.p.rapidapi.com",
      'x-rapidapi-key': rapid_api_key
      }

  # get property detail
  z_prop_detail_resp = requests.request("GET", url, headers=headers, params=querystring)
  z_prop_detail_resp_json = z_prop_detail_resp.json()

  # wait 1 sec based on limit
  time.sleep(1.5)

  prop_detail_list.append(z_prop_detail_resp_json)

In [12]:
# convert to dataframe
df_z_prop_detail = pd.json_normalize(prop_detail_list)
print('Num of rows:', len(df_z_prop_detail))
print('Num of cols:', len(df_z_prop_detail.columns))
df_z_prop_detail.head(2)

Num of rows: 1
Num of cols: 296


,listingProvider,zpid,buildingPermits,propertyTaxRate,contact_recipients,zipcode,openHouseSchedule,longitude,zestimateLowPercent,cityId,timeOnZillow,url,zestimate,imgSrc,description,price,livingAreaValue,taxHistory,brokerId,stateId,streetAddress,solarPotential,dateSold,countyId,timeZone,homeType,livingAreaUnits,comingSoonOnMarketDate,rentZestimate,bathrooms,annualHomeownersInsurance,state,propertyTypeDimension,building,yearBuilt,brokerageName,pageViewCount,county,monthlyHoaFee,homeStatus,homeFacts,latitude,datePosted,bedrooms,nearbyHomes,livingArea,priceHistory,favoriteCount,schools,zestimateHighPercent,mlsid,countyFIPS,city,providerListingID,country,currency,isListedByOwner,contingentListingType,address.community,address.city,address.state,address.neighborhood,address.subdivision,address.streetAddress,address.zipcode,resoFacts.hasAttachedProperty,resoFacts.frontageType,resoFacts.poolFeatures,resoFacts.flooring,resoFacts.builderModel,resoFacts.accessibilityFeatures,resoFacts.hasGarage,resoFacts.hasPetsAllowed,resoFacts.bodyType,resoFacts.topography,resoFacts.landLeaseExpirationDate,resoFacts.hasAdditionalParcels,resoFacts.waterViewYN,resoFacts.atAGlanceFacts,resoFacts.offerReviewDate,resoFacts.horseYN,resoFacts.buyerAgencyCompensationType,resoFacts.belowGradeFinishedArea,resoFacts.feesAndDues,resoFacts.cityRegion,resoFacts.subAgencyCompensation,resoFacts.hasPrivatePool,resoFacts.associationFeeIncludes,resoFacts.waterSource,resoFacts.carportParkingCapacity,resoFacts.transactionBrokerCompensationType,resoFacts.hasFireplace,resoFacts.gas,resoFacts.sewer,resoFacts.elevationUnits,resoFacts.roadSurfaceType,resoFacts.mainLevelBedrooms,resoFacts.numberOfUnitsVacant,resoFacts.hasWaterfrontView,resoFacts.bathroomsOneQuarter,resoFacts.lotSize,resoFacts.subAgencyCompensationType,resoFacts.entryLevel,resoFacts.irrigationWaterRightsAcres,resoFacts.greenWaterConservation,resoFacts.stories,resoFacts.bathrooms,resoFacts.numberOfUnitsInCommunity,resoFacts.listingTerms,resoFacts.otherParking,resoFacts.associationFee,resoFacts.marketingType,resoFacts.greenEnergyEfficient,resoFacts.greenSustainability,resoFacts.heating,resoFacts.associationPhone,resoFacts.greenBuildingVerificationType,resoFacts.hasAttachedGarage,resoFacts.bedrooms,resoFacts.architecturalStyle,resoFacts.listingId,resoFacts.structureType,resoFacts.interiorFeatures,resoFacts.horseAmenities,resoFacts.electric,resoFacts.lotFeatures,resoFacts.roofType,resoFacts.compensationBasedOn,resoFacts.greenEnergyGeneration,resoFacts.taxAnnualAmount,resoFacts.daysOnZillow,resoFacts.listAOR,resoFacts.hasHeating,resoFacts.elementarySchool,resoFacts.constructionMaterials,resoFacts.basementYN,resoFacts.hoaFeeTotal,resoFacts.appliances,resoFacts.virtualTour,resoFacts.bathroomsPartial,resoFacts.fencing,resoFacts.yearBuiltEffective,resoFacts.buildingName,resoFacts.hasAssociation,resoFacts.isSeniorCommunity,resoFacts.propertySubType,resoFacts.propertyCondition,resoFacts.hasCarport,resoFacts.canRaiseHorses,resoFacts.hasLandLease,resoFacts.communityFeatures,resoFacts.parcelNumber,resoFacts.elevation,resoFacts.lotSizeDimensions,resoFacts.buildingAreaSource,resoFacts.contingency,resoFacts.yearBuilt,resoFacts.otherStructures,resoFacts.livingArea,resoFacts.bathroomsFull,resoFacts.livingAreaRangeUnits,resoFacts.incomeIncludes,resoFacts.highSchool,resoFacts.aboveGradeFinishedArea,resoFacts.isNewConstruction,resoFacts.inclusions,resoFacts.hasHomeWarranty,resoFacts.additionalFeeInfo,resoFacts.windowFeatures,resoFacts.ownership,resoFacts.woodedArea,resoFacts.middleOrJuniorSchoolDistrict,resoFacts.associationPhone2,resoFacts.buildingFeatures,resoFacts.frontageLength,resoFacts.buildingArea,resoFacts.associationAmenities,resoFacts.bathroomsHalf,resoFacts.hasCooling,resoFacts.additionalParcelsDescription,resoFacts.parkingFeatures,resoFacts.foundationArea,resoFacts.zoning,resoFacts.hoaFee,resoFacts.livingAreaRange,resoFacts.zoningDescription,resoFacts.developmentStatus,resoFacts.availabilityDate,resoFacts.waterfrontFeatur

In [13]:
# columns of interest
detail_cols = ['streetAddress',
 'city',
 'county',
 'zipcode',
 'state',
 'price',
 'homeType',
 'timeOnZillow',
 'zestimate',
 'rentZestimate',
 'livingArea',
 'bedrooms',
 'bathrooms',
 'yearBuilt',
 'description',
 'priceHistory',
 'taxHistory',
 'zpid'
 ]

# retain limited columns for output
df_z_prop_detail_output = df_z_prop_detail[detail_cols]
df_z_prop_detail_output.head()

,streetAddress,city,county,zipcode,state,price,homeType,timeOnZillow,zestimate,rentZestimate,livingArea,bedrooms,bathrooms,yearBuilt,description,priceHistory,taxHistory,zpid
0,54 Miramonte Dr,Moraga,Contra Costa County,94556,CA,859000,CONDO,95 days,None,3951,1248,2,2,1964,INCREDIBLE PRICE!! This captivating two-story ...,"[{'priceChangeRate': -0.022753128555176, 'date...","[{'time': 1682541735080, 'valueIncreaseRate': ...",18471532


In [14]:
# download file
#df_z_prop_detail_output.to_csv('df_z_prop_detail_output.csv')
#files.download('df_z_prop_detail_output.csv')

## Zillow ZPID
Useful to retrieve zillow detailed data for ANY address

In [15]:
df_z_prop_detail_output['streetAddress'].iloc[0]

'54 Miramonte Dr'

In [16]:
data = """Name,Address
"""

In [17]:
# Step 1: Open the file in write mode
file = open('example.txt', 'w')

# Step 2: Write text to the file
file.write("Name,Address\n")
file.write("House_1, "+df_z_prop_detail_output['streetAddress'].iloc[0]+"; "+df_z_prop_detail_output['city'].iloc[0]+" CA"+"\n")
# Step 3: Close the file
file.close()

In [18]:
import pandas as pd
data=pd.read_csv('./example.txt')

In [23]:
df=data

In [24]:
import pandas as pd
import requests
import json
import urllib
import io

In [25]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="example app")

In [26]:
df["loc"] = df["Address"].apply(geolocator.geocode)

In [27]:
df["point"]= df["loc"].apply(lambda loc: tuple(loc.point) if loc else None)

In [28]:
df[['lat', 'lon', 'altitude']] = pd.DataFrame(df['point'].to_list(), index=df.index)

In [29]:
# import the library and its Marker clusterization service
import folium
from folium.plugins import MarkerCluster
# Create a map object and center it to the avarage coordinates to m
m = folium.Map(location=df[["lat", "lon"]].mean().to_list(), zoom_start=2)
# if the points are too close to each other, cluster them, create a cluster overlay with MarkerCluster, add to m
marker_cluster = MarkerCluster().add_to(m)
# draw the markers and assign popup and hover texts
# add the markers the the cluster layers so that they are automatically clustered
for i,r in df.iterrows():
    location = (r["lat"], r["lon"])
    folium.Marker(location=location,
                      popup = r['Name'],
                      tooltip=r['Name'])\
    .add_to(marker_cluster)
# display the map
m

In [ ]:
#https://towardsdatascience.com/pythons-geocoding-convert-a-list-of-addresses-into-a-map-f522ef513fd6#5352

# End Notebook